<a href="https://colab.research.google.com/github/Satish-970/ats_resume_checker/blob/main/ATS_Checker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
# Install required libraries
# Install required libraries and Java 17
!apt-get update -qq
!apt-get install -qq openjdk-17-jre-headless
!java -version  # Verify Java version



# Import libraries
import spacy
import PyPDF2
import requests
from bs4 import BeautifulSoup
import language_tool_python
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
from ipywidgets import FileUpload, Textarea, Text, Button, Output, VBox
from IPython.display import display
import re
import os

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Selecting previously unselected package openjdk-17-jre-headless:amd64.
(Reading database ... 126380 files and directories currently installed.)
Preparing to unpack .../openjdk-17-jre-headless_17.0.16+8~us1-0ubuntu1~22.04.1_amd64.deb ...
Unpacking openjdk-17-jre-headless:amd64 (17.0.16+8~us1-0ubuntu1~22.04.1) ...
Setting up openjdk-17-jre-headless:amd64 (17.0.16+8~us1-0ubuntu1~22.04.1) ...
update-alternatives: using /usr/lib/jvm/java-17-openjdk-amd64/bin/java to provide /usr/bin/java (java) in auto mode
update-alternatives: using /usr/lib/jvm/java-17-openjdk-amd64/bin/jpackage to provide /usr/bin/jpackage (jpackage) in auto mode
update-alternatives: using /usr/lib/jvm/java-17-openjdk-amd64/bin/keytool to provide /usr/bin/keytool (keytool) in auto mode
update-alternatives: using /usr/lib/jvm/java-17-op

In [2]:
def extract_resume_text(file_path):
    """
    Extracts text from a PDF or text file resume.
    """
    try:
        if file_path.endswith('.pdf'):
            with open(file_path, 'rb') as file:
                reader = PyPDF2.PdfReader(file)
                text = ''
                for page in reader.pages:
                    extracted = page.extract_text()
                    if extracted:
                        text += extracted + ' '
        elif file_path.endswith('.txt'):
            with open(file_path, 'r', encoding='utf-8') as file:
                text = file.read()
        else:
            return None, "Unsupported file format. Use PDF or TXT."
        return text.strip(), None
    except Exception as e:
        return None, f"Error reading file: {str(e)}"

In [3]:
def extract_job_description(input_text):
    """
    Extracts job description from text or URL.
    """
    try:
        if input_text.startswith(('http://', 'https://')):
            response = requests.get(input_text, timeout=10)
            response.raise_for_status()
            soup = BeautifulSoup(response.text, 'html.parser')
            # Target common job description containers (customize as needed)
            job_text = ''
            for tag in soup.find_all(['p', 'div', 'li']):
                if len(tag.get_text().split()) > 10:  # Filter short fragments
                    job_text += tag.get_text() + ' '
            return job_text.strip(), None
        else:
            return input_text.strip(), None
    except Exception as e:
        return None, f"Error processing job description: {str(e)}"

In [4]:
nlp = spacy.load('en_core_web_sm')  # Load spaCy model

def extract_keywords(text):
    """
    Extracts keywords (nouns, proper nouns, and custom skills) from text.
    """
    if not text:
        return set()
    doc = nlp(text.lower())
    keywords = set()
    custom_skills = {'python', 'sql', 'machine learning', 'data analysis', 'java', 'aws', 'tableau', 'tensorflow'}
    for token in doc:
        if token.pos_ in ['NOUN', 'PROPN'] or token.text in custom_skills:
            keywords.add(token.text)
    return keywords

In [5]:
def calculate_ats_score(resume_text, job_description):
    """
    Computes similarity score between resume and job description.
    """
    if not resume_text or not job_description:
        return 0.0
    documents = [resume_text, job_description]
    vectorizer = TfidfVectorizer(stop_words='english')
    tfidf_matrix = vectorizer.fit_transform(documents)
    similarity = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:2])[0][0]
    return similarity * 100

In [6]:
def get_suggestions(resume_keywords, job_keywords):
    """
    Finds keywords in job description not in resume.
    """
    missing = job_keywords - resume_keywords
    if missing:
        return f"Suggestions: Add {', '.join(sorted(missing))} to your resume for better alignment."
    return "Great match! No suggestions needed."

In [7]:
def check_grammar(text):
    """
    Checks grammar and spelling in the resume text.
    """
    tool = language_tool_python.LanguageTool('en-US')
    matches = tool.check(text)
    if not matches:
        return "No grammar or spelling issues found."
    suggestions = []
    for match in matches:
        suggestions.append(f"Issue: {match.message} at '{text[match.offset:match.offset+match.errorLength]}'. Suggestion: {match.replacements}")
    return "\n".join(suggestions)

In [8]:
def visualize_score(score):
    """
    Plots a bar chart of the ATS score.
    """
    plt.figure(figsize=(6, 4))
    plt.bar(['ATS Score'], [score], color='skyblue')
    plt.ylim(0, 100)
    plt.ylabel('Match Percentage')
    plt.title('Resume ATS Match Score')
    plt.show()

In [13]:

# Install required Python libraries
!pip install spacy pypdf2 scikit-learn language-tool-python requests beautifulsoup4 -qq

!python -m spacy download en_core_web_sm -qq

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [14]:
# Create widgets
upload = FileUpload(accept='.pdf,.txt', description='Upload Resume (PDF/TXT)')
job_desc = Text(placeholder='Paste job description or URL', description='Job Desc/URL:')
button = Button(description='Check ATS Score')
output = Output()

def on_button_clicked(b):
    with output:
        output.clear_output()
        if not upload.value:
            print("Please upload a resume (PDF or TXT).")
            return

        # Process uploaded resume
        try:
            uploaded_file = list(upload.value.values())[0]
            file_name = uploaded_file.get('metadata', {}).get('name', 'temp_resume')
            file_extension = os.path.splitext(file_name)[1] if file_name else '.pdf'
            temp_file_name = f'temp_resume{file_extension}'
            with open(temp_file_name, 'wb') as f:
                f.write(uploaded_file['content'])

            resume_text, resume_error = extract_resume_text(temp_file_name)
            if resume_error:
                print(f"Error processing resume: {resume_error}")
                return
        except Exception as e:
            print(f"Failed to process uploaded file: {str(e)}")
            return

        # Process job description
        job_input = job_desc.value.strip()
        if not job_input:
            print("No job description provided. Performing grammar check on resume.")
            print(check_grammar(resume_text))
            return

        job_text, job_error = extract_job_description(job_input)
        if job_error:
            print(f"Error processing job description: {job_error}")
            return

        # Calculate ATS score and suggestions
        resume_keywords = extract_keywords(resume_text)
        job_keywords = extract_keywords(job_text)
        ats_score = calculate_ats_score(resume_text, job_text)
        print(f"ATS Match Score: {ats_score:.2f}%")
        print(get_suggestions(resume_keywords, job_keywords))
        print("\nResume Grammar Check:")
        print(check_grammar(resume_text))
        visualize_score(ats_score)

button.on_click(on_button_clicked)
display(VBox([upload, job_desc, button, output]))